In [1]:
#import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import time
pd.options.display.max_rows = 10000
pd.options.display.max_seq_items = 10000
pd.set_option('display.max_colwidth', None)
from selenium import webdriver
import html5lib
import lxml
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time


### Campany Scrape

Company Scrape The first step is to pull companies that have a top work-life balance for a role you are interested in. For example, pull companies with a work-life balance >= 3.5 stars for Analyst roles. This scrape will pull a max of ~9,900 companies as Glassdoor has a bug that doesn't show more companies after page 1,000 on the company explore section of the site.

In [3]:
#create temp df
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
length = 1
i = 1
#while statement will go to 1000 as Glassdoor doesn't share companies
#after page 1000
while (i <= 1000):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    #scrape companies with DS jobs with WLB > 4.5 stars
    #url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=4.5&page=" + str(i) + "&occ=Data%20Scientist&filterType=RATING_WORK_LIFE_BALANCE"
    
    #scrape companies with Analyst jobs with WLB > 3.5 stars
    url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=3.5&page=" + str(i) + "&occ=Analyst&filterType=RATING_WORK_LIFE_BALANCE"
    
    #scrape companies with overall WLB > 4.5 stars
    #url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=4.5&page=" + str(i) + "&filterType=RATING_WORK_LIFE_BALANCE"
    
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    #table = soup.find_all('div', {'class':'col-md-8'})
    #table = soup.find_all('div', {'class':'mt-0 mb-std p-std css-1ax1pfu css-errlgf'})
    #table = soup.find_all('div', {'class':'row d-flex flex-wrap'})


    
    table = soup.find_all('h2', {'data-test':'employer-short-name'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    l=[]
    for link in soup.find_all('a',{'data-test':'cell-Jobs-url'}):
     ##   print(link.get('href'))
        l.append(link.get('href')) 
    employees = soup.find_all('span',{'data-test':'employer-size'})
    #print(employees)
    #print(job)
    length=len(table)
    #if (len(table) == 0):
    #    break;

    #print(table)
    y = pd.DataFrame(l)
    #print(y)
    x = pd.DataFrame(table)
    z = pd.DataFrame(employees)
    df = df.append(x)
    df2 = df2.append(y)
    df3 = df3.append(z)
    #print(len(df3))
    #print(i)
    i = i + 1


10
1
20
2
30
3
40
4
50
5
60
6
70
7
80
8
90
9
100
10
110
11
120
12
130
13
140
14
150
15
160
16
170
17
180
18
190
19
200
20
210
21
220
22
230
23
240
24
250
25
260
26
270
27
280
28
290
29
300
30
310
31
320
32
330
33
340
34
350
35
360
36
370
37
380
38
390
39
400
40
410
41
420
42
430
43
440
44
450
45
460
46
470
47
480
48
490
49
500
50
510
51
520
52
530
53
540
54
550
55
560
56
570
57
580
58
590
59
600
60
610
61
620
62
630
63
640
64
650
65
660
66
670
67
680
68
690
69
700
70
710
71
720
72
730
73
740
74
750
75
760
76
770
77
780
78
790
79
800
80
810
81
820
82
830
83
840
84
850
85
860
86
870
87
880
88
890
89
900
90
910
91
920
92
930
93
940
94
950
95
960
96
970
97
980
98
990
99
1000
100
1010
101
1020
102
1030
103
1040
104
1050
105
1060
106
1070
107
1080
108
1090
109
1100
110
1110
111
1120
112
1130
113
1140
114
1150
115
1160
116
1170
117
1180
118
1190
119
1200
120
1210
121
1220
122
1230
123
1240
124
1250
125
1260
126
1270
127
1280
128
1290
129
1300
130
1310
131
1320
132
1330
133
1340
134
1350
135
1

9350
936
9360
937
9370
938
9380
939
9390
940
9400
941
9410
942
9420
943
9430
944
9440
945
9450
946
9460
947
9470
948
9480
949
9490
950
9500
951
9510
952
9520
953
9530
954
9540
955
9550
956
9560
957
9570
958
9580
959
9590
960
9600
961
9610
962
9620
963
9630
964
9640
965
9650
966
9660
967
9670
968
9680
969
9690
970
9700
971
9710
972
9720
973
9730
974
9740
975
9750
976
9760
977
9770
978
9780
979
9790
980
9800
981
9810
982
9820
983
9830
984
9840
985
9850
986
9860
987
9870
988
9880
989
9890
990
9900
991
9910
992
9920
993
9930
994
9940
995
9950
996
9960
997
9970
998
9980
999
9980
1000


In [4]:
#clean up pulled data
df.columns=['company']
df2.columns=['job_url']
df3.columns=['employees']
df4 = pd.concat([df,df2,df3],axis=1, ignore_index=True)
#df3 = pd.DataFrame(df,df2)
df4.columns=['company','job_url','employees']
df4=df4.reset_index()

In [5]:
#saving checkpoint - save list of companies pulled
df4.to_csv('top_worklife_companies_analyst_09162023.csv')

In [6]:
#load in saved file
df4 = pd.read_csv('top_worklife_companies_analyst_09162023.csv')

In [7]:
df4.shape

(9980, 5)

### Job Scrape

Job Scrape from pulled companies From each of the companies scraped, the scraper will go to their company site and pull their current job openings. A call out here is that if the company has more than 40 job openings, only the first 40 jobs will be pulled as glassdoor limits the amount of jobs shown on the company profile page.

In [8]:
df5 = pd.DataFrame()
df6 = pd.DataFrame()
#df7 = pd.DataFrame()
company_list = []
j= 1
length = 1
for i in range(0,len(df4)):

    x = df4['job_url'][i]
    x = x[:-4]
    x = x.split("-Jobs-")

    job_url='https://www.glassdoor.com' + x[0] + "-Remote-Jobs-" + x[1] + "_P" + str(j) + ".htm"
   # print(job_url)
    company = df4['company'][i]
  #  print(company)

    driver = webdriver.Chrome(ChromeDriverManager().install())#, chrome_options=options)
    driver.get(job_url)

    html = driver.page_source
    #print(html)
    soup = BeautifulSoup(html, "lxml")
   # print(soup)
    job_title = soup.find_all('div', {'class':'job-title'})
    job_location = soup.find_all('div',{'class':'css-k7ss21'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    l=[]
    for link in soup.find_all('a',{'data-test':'job-link'}):
    #    print(link.get('href'))
        l.append(link.get('href')) 
        company_list.append(company)

    #print(job)
    #print(job_title)
    length=len(job_title)
   # print(length)
    if (len(job_title) == 0):
        j = 1
    else:
        j = j + 1

    #print(table)
    y = pd.DataFrame(l)
    #print(y)
    x = pd.DataFrame(job_title)
    z = pd.DataFrame(job_location)
    #employee_cnt = pd.DataFrame()
    df5 = df5.append(x)
    df6 = df6.append(y)


KeyboardInterrupt: 

In [9]:
#company_list
df6.shape

(67961, 1)

In [10]:
#clean up job dataframe pulled
df5.columns=['job name']
df6.columns=['job link']
df7 = pd.concat([df5,df6],axis=1, ignore_index=True)
#df3 = pd.DataFrame(df,df2)
df7.columns=['job_name','job_link']
df7=df7.reset_index()
df7['company'] = company_list

In [11]:
df8 = pd.merge(df7, df4, how = 'inner', on = 'company')
df8 = df8.drop(columns =['index_y','index_x','job_url'])
#df8

In [12]:
#filter out jobs to only include certain keywords in job title
df9 = df8[(df8['job_name'].str.contains('Analyst'))|
         (df8['job_name'].str.contains('Data')) |
         (df8['job_name'].str.contains('Scientist'))]

df9.shape

(7156, 5)

In [13]:
df9= df9.reset_index()

In [14]:
#saving checkpoint - save jobs df
df9.to_csv('job_scrape_job_links_analyst_09162023_p1.csv')

In [15]:
df9 = pd.read_csv('job_scrape_job_links_analyst_09162023_p1.csv')

In [16]:
df9.head()

,Unnamed: 0.1,index,job_name,job_link,company,Unnamed: 0,employees
0,0,0,2024 – Data Science Analyst Program – Internship,/partner/jobListing.htm?pos=101&ao=1312534&s=21&guid=0000018a9e205edabba30364b0137654&src=GD_JOB_AD&t=ESR&vt=w&cs=1_767f7389&cb=1694870036442&jobListingId=1008829185595&jrtk=5-yul1-0-1haf20npuk5rs807-61ab16103865d211,JPMorgan Chase & Co,0,10000+ Employees
1,1,4,2024 Chase Leadership Development Program - Summer Analyst Opportunity,/partner/jobListing.htm?pos=105&ao=1312534&s=21&guid=0000018a9e205edabba30364b0137654&src=GD_JOB_AD&t=ESR&vt=w&cs=1_7ae85709&cb=1694870036442&jobListingId=1008812368701&jrtk=5-yul1-0-1haf20npuk5rs807-7de0e6a817d01ea6,JPMorgan Chase & Co,0,10000+ Employees
2,2,10,Portfolio Quality Analyst,/partner/jobListing.htm?pos=111&ao=1312534&s=21&guid=0000018a9e205edabba30364b0137654&src=GD_JOB_AD&t=ESR&vt=w&cs=1_6098ef56&cb=1694870036442&jobListingId=1008860225454&jrtk=5-yul1-0-1haf20npuk5rs807-43c6596151d17281,JPMorgan Chase & Co,0,10000+ Employees
3,3,18,Portfolio Quality Analyst,/partner/jobListing.htm?pos=119&ao=1312534&s=21&guid=0000018a9e205edabba30364b0137654&src=GD_JOB_AD&t=ESR&vt=w&cs=1_f3a8fbd9&cb=1694870036443&jobListingId=1008842855158&jrtk=5-yul1-0-1haf20npuk5rs807-9486a8dd3e869ecb,JPMorgan Chase & Co,0,10000+ Employees
4,4,25,2024 AI & Data Science Summer Associate Program,/partner/jobListing.htm?pos=126&ao=1312534&s=21&guid=0000018a9e205edabba30364b0137654&src=GD_JOB_AD&t=ESR&vt=w&cs=1_639877d9&cb=1694870036445&jobListingId=1008772097451&jrtk=5-yul1-0-1haf20npuk5rs807-e20df50e9935fcaa,JPMorgan Chase & Co,0,10000+ Employees


### Examine Job Description

Examine jobs scraped and filter out those that don't fit your search criteria and see which of those are open to WFH policy.

In [16]:
#df9 = pd.DataFrame()
temp_l = []
#might need to run this iteratively depending on the amount of jobs 
#you pulled and the memory on your pc. 
#Keeping it iteration to around 1000 jobs is recommended
df10 = df9.head(1102)
for i in range(0,len(df10)):
    
    print(i)
    job_url2 = 'http://glassdoor.com' + df10['job_link'][i]
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(job_url2)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    #job_desc = soup.find_all('div', {'class':'desc css-58vpdc ecgq1xb5'})
    job_desc = soup.find_all('div', {'class':'desc css-c43yjn ecgq1xb5'})

    #print(job_desc)
    #if (job_desc.isempty()):
    job_desc2 = soup.find_all('div', {'class':'css-15k9b95 elvm8840'})

    #job_desc = soup.find_all('div', {'class':'css-15k9b95 elvm8840'})
    #print(job_desc)
    remote_check = str(job_desc).find('remote')
    remote_check2 = str(job_desc).find('WFH')
    remote_check3 = str(job_desc).find('home')
    remote_check4 = str(job_desc2).find('remote')
    remote_check5 = str(job_desc2).find('WFH')
    remote_check6 = str(job_desc2).find('home')
        
    if ((remote_check > 0) | 
        (remote_check2 > 0 ) |
        (remote_check3 > 0) |
        (remote_check4 > 0 ) |
        (remote_check5 > 0) |
        (remote_check6 > 0) 
       ):
        remote_final = 1
    else:
        remote_final = -1
    #temp_df = pd.DataFrame(remote_final)

    temp_l.append(remote_final)
    #df9 = df9.append(temp_df)
    
    
    

#df9.columns = ['remote_check']
df10['remote_check'] = temp_l#df9['remote_check']
df10.to_csv('job_scrape_final_09122023_p6.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232


In [ ]:
#END